# Init

In [25]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import glob
from scipy.spatial.distance import euclidean, pdist, squareform
import sys
import sys
import os
sys.path.append(os.environ['DEV_AUTOTS'])
sys.path.append(os.environ['CAPSTONE_PYTHON_SOURCE'])
folder = os.environ['CAPSTONE_DATA']


In [26]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from ETL.ETL import loadDataset, getTopProducts
from similarity.similarity import mergeTopSimilar, loadSimilarity
dataRaw= loadDataset(version=3)
dfSimilarity = loadSimilarity(version=3)


In [28]:
#Parameters
ChainMaster = 'INDEPENDENTS'
ProdCat='SUP PREM WHISKEY'
#Customer = 'TWIN PEAKS LIQUOR'
prods = getTopProducts(dataRaw,ChainMaster=ChainMaster,ProdCat=ProdCat,topN=5,timeCol='Month')
prods

['JACK DANIELS BLK WHSKY  1L',
 'JACK DANIELS BLK WHSKY  750M',
 'JACK DANIELS BLK WHSKY  1.75L',
 'JACK DANIELS BLK WHSKY FLSK PET 375M',
 'JACK DANIELS BLK WHSKY RD 375M']

In [29]:

prod = prods[0]
colTime = 'Month'
colPreds = ['9L Cases'] #Demand
colEndog = ['Dollar Sales per 9L Case'] #Price

#add function to get the top products ##DONBE
#create encoder/decoder for the product names ##DONE
#add filter only for the category of product. DONE
#add top nproduct based on avg revenue  per month for full dataset   calculate since the first month of product.. limit at least for 24 months DONE
#REMOVE current product in the pivot Done
#
#TO DO:
#Merge top similar only looking at a list of products 
#Encode before the merge? (same prods same code)
#one chain, top three products, top 1 similar
#Impute the price
(dataModel,colExog,colEnc,colDec) = mergeTopSimilar(dataRaw,dfSimilarity
                                                    ,ChainMaster=ChainMaster
                                                    ,Product=prod
                                                    ,topn=5 
                                                    ,selCols = colEndog + colPreds,periodCol = colTime
                                                    ,pivotCol='Dollar Sales per 9L Case' 
                                                    ,encodeCols=True)
                                                                        
#colExog = colExog + colEndog
print(colExog)
print(colDec)
allCols=[colTime]+colPreds+colEndog + colExog
data=dataModel[allCols]
data.tail(5)

['0', '1', '2', '3']
{'0': 'JACK DANIELS BLK WHSKY  1.75L', '1': 'JACK DANIELS GRN  1L', '2': 'JACK DANIELS BLK WHSKY 3PK W/2L COKE 1.75L', '3': 'JACK DANIELS GRN  1.75L'}


,Month,9L Cases,Dollar Sales per 9L Case,0,1,2,3
79,2019-08-01,40.898,3586.205705,1768.153846,217.984962,1770.310345,168.256410
80,2019-09-01,28.264,3584.534954,1432.256410,0.000000,603.232759,178.512821
81,2019-10-01,36.687,3823.402560,1626.433048,0.000000,2018.803448,179.538462
82,2019-11-01,40.011,4302.033466,1768.153846,217.984962,0.000000,0.000000
83,2019-12-01,44.112,4063.029034,3333.435897,0.000000,0.000000,168.256410


In [30]:
train = data.iloc[0:80]
test = data.iloc[80:84]
print(train.shape,test.shape)

(80, 7) (4, 7)


In [31]:
import auto_ts as AT
automl_model = AT.AutoTimeSeries(
    score_type='rmse', forecast_period=3, time_interval='Month',
    non_seasonal_pdq=None, seasonality=True, seasonal_period=12,
    model_type=['SARIMAX','ML'],
    verbose=0)
automl_model.fit(train, 'Month', '9L Cases', ',')



Start of Fit.....
    Target variable = 9L Cases
Start of loading of data.....
Input is data frame. Performing Time Series Analysis
    Dataframe loaded successfully. Shape of data set = (80, 6)
Time Interval is given as Month


Building SARIMAX Model



Running Seasonal SARIMAX Model...
Building a Seasonal Model...

    Finding best Non-Seasonal pdq Parameters:

Differencing = 0 with Seasonality = False
    Iteration 0 completed...
Inside find_best_pdq_or_PDQ --> results_bic
            MA0         MA1         MA2         MA3
AR0         NaN  522.423505  515.944606  506.922266
AR1  527.709657  520.863435  516.521451  508.920373
AR2  521.865392  522.702711  518.150545  510.396229
AR3  516.733736  516.420236  518.610746  510.571151
    Best AR order p = 0, MA order q = 3, Interim metric = 506.922

Differencing = 1 with Seasonality = True
Inside find_best_pdq_or_PDQ --> results_bic
     MA0  MA1  MA2  MA3
AR0  NaN  NaN  NaN  NaN
AR1  NaN  NaN  NaN  NaN
AR2  NaN  NaN  NaN  NaN
AR3  NaN  N

In [32]:
automl_model.get_leaderboard()

,name,rmse
1,ML,7.772846
0,SARIMAX,8.362560


In [33]:
train.tail(3)

,Month,9L Cases,Dollar Sales per 9L Case,0,1,2,3
77,2019-06-01,33.694,4542.229576,1807.897436,0.000000,1209.568966,354.461538
78,2019-07-01,30.812,3584.489661,2669.076923,217.984962,623.172414,0.000000
79,2019-08-01,40.898,3586.205705,1768.153846,217.984962,1770.310345,168.256410


In [34]:
automl_model.get_ml_dict()

defaultdict(<function auto_ts.AutoTimeSeries.fit.<locals>.<lambda>()>,
            {'SARIMAX': defaultdict(<function auto_ts.AutoTimeSeries.fit.<locals>.<lambda>()>,
                         {'model': <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper at 0x1b051ad63c8>,
                          'forecast': 9L Cases         mean   mean_se  mean_ci_lower  mean_ci_upper
                          2019-06-01  44.042419  5.415340      33.428548      54.656291
                          2019-07-01  29.879717  5.416237      19.264088      40.495346
                          2019-08-01  30.806504  5.514944      19.997412      41.615596,
                          'rmse': 8.362560089146708,
                          'model_build': <auto_ts.models.ar_based.sarimax.BuildSarimax at 0x1b051419e48>}),
             'ML': defaultdict(<function auto_ts.AutoTimeSeries.fit.<locals>.<lambda>()>,
                         {'model': BaggingRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
  

In [35]:
test_predictions = automl_model.predict(X_exogen = test)  # TODO: pass the test dataframe (exogen) - can add model = to to get a model speicifc
test_predictions

0    29.63932
1    29.19418
2    36.77458
3    28.79758
Name: mean, dtype: float64

In [36]:
test_predictions = automl_model.predict(X_exogen = test,model='ML')  # TODO: pass the test dataframe (exogen) - can add model = to to get a model speicifc
print(test_predictions)
test_predictions = automl_model.predict(X_exogen = test[colEndog+colExog ],model='SARIMAX')  # TODO: pass the test dataframe (exogen) - can add model = to to get a model speicifc
print(test_predictions)
test

0    29.63932
1    29.19418
2    36.77458
3    28.79758
Name: mean, dtype: float64
2019-09-01    31.610058
2019-10-01    35.713491
2019-11-01    45.651439
2019-12-01    56.811153
Freq: MS, Name: mean, dtype: float64


,Month,9L Cases,Dollar Sales per 9L Case,0,1,2,3
80,2019-09-01,28.264,3584.534954,1432.256410,0.000000,603.232759,178.512821
81,2019-10-01,36.687,3823.402560,1626.433048,0.000000,2018.803448,179.538462
82,2019-11-01,40.011,4302.033466,1768.153846,217.984962,0.000000,0.000000
83,2019-12-01,44.112,4063.029034,3333.435897,0.000000,0.000000,168.256410


## Predicting with Exogenous Vars

In [51]:
#create fake prediction dataset
dataPred = data.copy()
dataPred.index = dataPred[colTime]
dataPred=dataPred.asfreq(freq=pd.offsets.MonthBegin())


dfExog= pd.DataFrame({'Month' : pd.date_range('2020-01-01',periods = 3,freq='MS')})
cols= colEndog + colExog
for i,c in enumerate(cols):
    dfExog[c] = dataPred[c][-1] 
    dfExog[c]  = dfExog[c] ** (1+dfExog.index/500)

dfExog.set_index(colTime,inplace=True)
dfExog=dfExog.asfreq(freq=pd.offsets.MonthBegin())
dfExog


,Dollar Sales per 9L Case,TAAKA VODKA 80 PET 1.75L,TAAKA VODKA 80 BLUE PET 375M,TAAKA VODKA 80 BLUE PET 200M,TAAKA CAKE VODKA PET 1.75L,DOBRA VODKA PET 375M
Month,,,,,,
2020-01-01,464.564103,464.564103,755.523189,188.411215,0.0,0.0
2020-02-01,470.305156,470.305156,765.604177,190.395624,0.0,0.0
2020-03-01,476.117156,476.117156,775.819676,192.400934,0.0,0.0


In [57]:
test_predictions = automl_model.predict(forecast_period=3,model='ML',X_exogen=dfExog) 
print(test_predictions)

[49.62486486 49.62486486 49.62486486]
